In [122]:
import numpy as np
from svg.path import parse_path
from svg.path.path import Line, Arc, QuadraticBezier, CubicBezier
import svgpathtools
from xml.dom import minidom

In [85]:
ex_svg = "data/svg/airplane/1.svg"

# read the SVG file
doc = minidom.parse(ex_svg)
path_strings = [path.getAttribute('d') for path
                in doc.getElementsByTagName('path')]
doc.unlink()

# print the line draw commands
splines = []
for path_string in path_strings:
    path = parse_path(path_string)
    # print(path_string, len(path))
    for e in path:
        if isinstance(e, (Line,Arc,QuadraticBezier,CubicBezier)):
            x0 = e.start.real
            y0 = e.start.imag
            x1 = e.end.real
            y1 = e.end.imag
            print("(%.2f, %.2f) - (%.2f, %.2f)" % (x0, y0, x1, y1))
            splines.append(e)

(167.00, 268.00) - (172.00, 262.00)
(172.00, 262.00) - (183.00, 259.00)
(183.00, 259.00) - (240.00, 259.00)
(240.00, 259.00) - (253.00, 268.00)
(253.00, 268.00) - (265.00, 283.00)
(171.00, 264.00) - (149.00, 333.00)
(149.00, 333.00) - (148.00, 375.00)
(148.00, 375.00) - (154.00, 386.00)
(154.00, 386.00) - (183.00, 386.00)
(183.00, 386.00) - (222.00, 359.00)
(222.00, 359.00) - (246.00, 326.00)
(246.00, 326.00) - (255.00, 304.00)
(255.00, 304.00) - (268.00, 282.00)
(168.00, 272.00) - (153.00, 261.00)
(153.00, 261.00) - (108.00, 245.00)
(108.00, 245.00) - (84.00, 232.00)
(84.00, 232.00) - (47.00, 209.00)
(47.00, 209.00) - (9.00, 179.00)
(265.00, 285.00) - (337.00, 286.00)
(337.00, 286.00) - (358.00, 279.00)
(358.00, 279.00) - (376.00, 267.00)
(9.00, 179.00) - (22.00, 175.00)
(22.00, 175.00) - (48.00, 175.00)
(10.00, 180.00) - (27.00, 173.00)
(27.00, 173.00) - (59.00, 171.00)
(59.00, 171.00) - (214.00, 172.00)
(204.00, 173.00) - (261.00, 174.00)
(261.00, 174.00) - (329.00, 190.00)
(204.00,

In [137]:
from typing import Union
def interp(t, a, b):
    return a + (t) * (b-a)

def export_to_svg(paths,outputName):
    paths2 = []
    for path in paths:
        if isinstance(path,Line):
            paths2.append(svgpathtools.Line(path.start,path.end))
        elif isinstance(path,QuadraticBezier):
            paths2.append(svgpathtools.QuadraticBezier(path.start,path.control,path.end))
        elif isinstance(path,CubicBezier):
            paths2.append(svgpathtools.CubicBezier(path.start,path.control1,path.control2,path.end))
    svgpathtools.wsvg(paths2, filename=outputName)

def slice_center(path: Union[Line, Arc, QuadraticBezier, CubicBezier], start: float, end: float):
    """Slices the path from start to end (between 0 and 1). Returns two paths"""
    if isinstance(path,Line):
        point1 = (path.start.real+start*(path.end.real-path.start.real),path.start.imag+start*(path.end.imag-path.start.imag))
        point2 = (path.start.real+end*(path.end.real-path.start.real),path.start.imag+end*(path.end.imag-path.start.imag))
        pathA = Line(path.start,(point1[0]+point1[1]*1j))
        pathB = Line((point2[0]+point2[1]*1j),path.end)
        return pathA,pathB
    elif isinstance(path,Arc):
        return "Arc not supported :("
    elif isinstance(path,QuadraticBezier):
        return sp(path, start), split_quadratic_bezier(path, end, rev=True)
    elif isinstance(path,CubicBezier):
        return split_cubic_bezier(path, start), split_cubic_bezier(path, end, rev=True)

def split_quadratic_bezier(path: QuadraticBezier, t: float, rev=False):
    points = [path.start, path.control1, path.end]
    if rev:
        points = reversed(points)
        t = 1-t
    a, b, c = points
    d = interp(t, a, b)
    e = interp(t, b, c)
    f = interp(t, d, e)
    return QuadraticBezier(start=a, control1=d, end=f, relative=path.relative, smooth=path.smooth)

def split_cubic_bezier(path: CubicBezier, t: float, rev=False):
    points = [path.start, path.control1, path.control2, path.end]
    if rev:
        points = reversed(points)
        t = 1-t
    a, b, c, d = points
    e = interp(t, a, b)
    f = interp(t, b, c)
    g = interp(t, c, d)
    h = interp(t, e, f)
    j = interp(t, f, g)
    k = interp(t, h, j)
    # assert abs(k - path.point(t)) < 1e-8, "k should be " + str(path.point(t)) + " but is " + str(k)
    return CubicBezier(start=a, control1=e, control2=h, end=k, relative=path.relative, smooth=path.smooth)

def compare_sliced(path, sliced, t):
    return sum(np.absolute(sliced.point(np.linspace(0, 1)) - path.point(np.linspace(0, t))))

In [143]:
#Big Run
with open("data/svg/filelist.txt","r") as f:
    files = f.readlines()
for file in files:
    splines = []
    if file.strip()=="cigarette/4262.svg":
        continue
    doc = minidom.parse(f"data/svg/{file.strip()}")
    path_strings = [path.getAttribute('d') for path in doc.getElementsByTagName('path')]
    doc.unlink()
    for i,path_string in enumerate(path_strings):
        path = parse_path(path_string)
        for e in path:
            if isinstance(e,(Line,CubicBezier)):
                pathA,pathB = slice_center(e,0.3,0.7)
                splines.append(pathA)
                splines.append(pathB)
    export_to_svg(splines,f"data/svg2/{file.strip()}")
    break

[(167+268j), (167+268j), (169.3665+263.4813j), (172+262j), (167+268j), (167.70995+266.64439j), (170.15655+263.03691j), (167.212985+267.593317j), (168.44393+265.562146j), (167.5822685+266.9839657j)]
[(172+262j), (169.3665+263.4813j), (167+268j), (167+268j), (171.20995+262.44439j), (168.65655+264.83691j), (167+268j), (170.44393+263.162146j), (168.159585+265.785837j), (169.7586265+263.9492533j)]
[(172+262j), (175.9896+259.7559j), (177.6387+259.2365j), (183+259j), (173.19688+261.32677j), (176.48433+259.60008j), (179.24709000000001+259.16555j), (174.183115+260.808763j), (177.31315800000002+259.469721j), (175.1221279+260.4070504j)]
[(183+259j), (177.6387+259.2365j), (175.9896+259.7559j), (172+262j), (181.39161+259.07095j), (177.14397+259.39232j), (174.79272+260.42913j), (180.11731799999998+259.16736099999997j), (176.438595+259.70336299999997j), (179.0137011+259.3281616j)]
[(183+259j), (210.4512+257.7889j), (216.727+256.0078j), (240+259j), (191.23536000000001+258.63667j), (212.33394+257.25457